In [1]:
import os

In [2]:
%pwd

'c:\\Project\\Insurance-ml-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Project\\Insurance-ml-Project'

In [5]:
from dataclasses import dataclass, field
from typing import Dict
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [7]:

from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [9]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pathlib import Path
import joblib
import json

In [10]:
class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = mean_squared_error(actual, pred, squared=False)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        predicted_qualities = model.predict(test_x)

        rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        with open(self.config.metric_file_name, 'w') as file:
            json.dump(scores, file)

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e


[2024-03-13 18:04:40,910: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-13 18:04:40,919: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-13 18:04:40,925: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-13 18:04:40,937: INFO: common: created directory at: artifacts]
[2024-03-13 18:04:40,944: INFO: common: created directory at: artifacts/model_evaluation]


c:\Project\Insurance-ml-Project\venv\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
